In [1]:
%tensorflow_version 2.x
import tensorflow
import tensorflow.keras as keras
import tensorflow.keras.utils
from tensorflow.keras.layers import LSTM, Input, Dense, TimeDistributed, Masking, Bidirectional
from tensorflow.keras.models import Model

In [2]:
tensorflow.__version__

'2.3.0'

# General

In [3]:
alphabet = list("0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'.,-()/") # %:;&#
alphabet.insert(0, chr(0)) # '\x00' character, i.e., ord(0) to label concatenate

# Model

In [4]:
timesteps = None
input_features = 3
lstm_output = 1024
alphabet_output_size = len(alphabet)
eoc_output_size = 1
eow_output_isze = 1

In [5]:
lstm_input = Input(shape=(timesteps, input_features), name='stroke_input')
masking_layer =  Masking(mask_value=0., input_shape=(timesteps, input_features))(lstm_input)
lstm_layer = Bidirectional(LSTM(lstm_output, return_sequences=True))(masking_layer)

alphabet_output = TimeDistributed(Dense(alphabet_output_size, activation='softmax'), name='alphabet_output')(lstm_layer)
eoc_output = TimeDistributed(Dense(eoc_output_size, activation='sigmoid'), name='eoc_output')(lstm_layer)
eow_output = TimeDistributed(Dense(eow_output_isze, activation='sigmoid'), name='eow_output')(lstm_layer)

In [6]:
losses ={
    'alphabet_output': 'sparse_categorical_crossentropy',
    'eoc_output' : 'mean_squared_error',
    'eow_output' : 'mean_squared_error'
}

In [7]:
model = Model(inputs=[lstm_input], outputs=[alphabet_output, eoc_output, eow_output])
model.compile(loss=losses, optimizer='adam', metrics=['accuracy'])

# Preprocessing

In [8]:
!ls

best_model.hdf5  drive	sample_data


In [9]:
import numpy as np

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
training_dataset = np.load('/content/drive/My Drive/deepwriting/deepwriting_training.npz', allow_pickle=True)
validation_dataset = np.load('/content/drive/My Drive/deepwriting/deepwriting_validation.npz', allow_pickle=True)

In [ ]:
[key for key in training_dataset.keys()]
training_dataset['eow_labels']

In [13]:
def pad_to_length(sequences, max_len, two_dimensional=True):
    padded_sequence = []
    for seq in sequences:
        pad_len = max_len - len(seq)
        if two_dimensional:
            padded_seq = np.pad(seq, [(0, pad_len), (0, 0)], mode='constant', constant_values=0)
        else:
            padded_seq = np.pad(seq, (0, pad_len), mode='constant', constant_values=0)
            padded_seq = np.expand_dims(padded_seq, axis=1)
        padded_sequence.append(padded_seq)
    # check whether all lists have actually the same length
    assert len(list(filter(lambda x: x != max_len, [len(seq) for seq in padded_sequence]))) == 0
    return np.array(padded_sequence)

In [14]:
def check_lengths(seq, max_len):
    other_max_length = len(max(seq, key=len))
    assert other_max_length == max_len

## Pad Training input

In [15]:
max_len_training = len(max(training_dataset['strokes'], key=len))
max_len_validation = len(max(validation_dataset['strokes'], key=len))

In [16]:
training_input_padded = pad_to_length(training_dataset['strokes'], max_len_training)
validation_input_padded = pad_to_length(validation_dataset['strokes'], max_len_validation)

## Pad char labels

In [17]:
check_lengths(training_dataset['char_labels'], max_len_training)
check_lengths(validation_dataset['char_labels'], max_len_validation)

In [18]:
training_char_labels_padded = pad_to_length(training_dataset['char_labels'], max_len_training, False)
validation_char_labels_padded = pad_to_length(validation_dataset['char_labels'], max_len_validation, False)

In [19]:
classes = len(training_dataset['alphabet'])
#char_labels_padded = tensorflow.keras.utils.to_categorical(char_labels_padded, num_classes=classes)

## Pad EOC labels

In [20]:
check_lengths(training_dataset['eoc_labels'], max_len_training)
check_lengths(validation_dataset['eoc_labels'], max_len_validation)

In [21]:
training_eoc_labels_padded = pad_to_length(training_dataset['eoc_labels'], max_len_training, False)
validation_eoc_labels_padded = pad_to_length(validation_dataset['eoc_labels'], max_len_validation, False)

In [22]:
check_lengths(training_dataset['eow_labels'], max_len_training)
check_lengths(validation_dataset['eow_labels'], max_len_validation)

In [23]:
training_eow_labels_padded = pad_to_length(training_dataset['eow_labels'], max_len_training, False)
validation_eow_labels_padded = pad_to_length(validation_dataset['eow_labels'], max_len_validation, False)

# Training

In [24]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [25]:
monitor = 'val_alphabet_output_accuracy'
mode='max'

modelCheckpoint = ModelCheckpoint('./best_model.hdf5', monitor=monitor, save_best_only='True', mode=mode)
earlyStopping = EarlyStopping(monitor=monitor, patience=10, mode=mode)
reduceLROnPlateau = ReduceLROnPlateau(monitor=monitor, factor=0.1, patience=4, mode=mode)

In [ ]:
model.fit(
    {
        'stroke_input': training_input_padded
    },
    {
        'alphabet_output': training_char_labels_padded,
        'eoc_output': training_eoc_labels_padded,
        'eow_output': training_eow_labels_padded
    },
    callbacks=[modelCheckpoint, earlyStopping, reduceLROnPlateau],
    epochs=100,
    validation_data=(
        {
            'stroke_input': validation_input_padded
        },
        {
            'alphabet_output': validation_char_labels_padded,
            'eoc_output': validation_eoc_labels_padded,
            'eow_output': validation_eow_labels_padded
        }
    )
)

Epoch 1/100
1081/1081 [==============================] - 237s 220ms/step - loss: 0.8077 - alphabet_output_loss: 0.7933 - eoc_output_loss: 0.0110 - eow_output_loss: 0.0033 - alphabet_output_accuracy: 0.5929 - eoc_output_accuracy: 0.9771 - eow_output_accuracy: 0.9935 - val_loss: 0.3853 - val_alphabet_output_loss: 0.3775 - val_eoc_output_loss: 0.0064 - val_eow_output_loss: 0.0014 - val_alphabet_output_accuracy: 0.8328 - val_eoc_output_accuracy: 0.9876 - val_eow_output_accuracy: 0.9976
Epoch 2/100
1081/1081 [==============================] - 235s 218ms/step - loss: 0.2629 - alphabet_output_loss: 0.2572 - eoc_output_loss: 0.0048 - eow_output_loss: 9.1957e-04 - alphabet_output_accuracy: 0.8687 - eoc_output_accuracy: 0.9889 - eow_output_accuracy: 0.9980 - val_loss: 0.2722 - val_alphabet_output_loss: 0.2661 - val_eoc_output_loss: 0.0052 - val_eow_output_loss: 9.4353e-04 - val_alphabet_output_accuracy: 0.8804 - val_eoc_output_accuracy: 0.9899 - val_eow_output_accuracy: 0.9982
Epoch 3/100
 252/1

In [ ]:
training_dataset['strokes']

In [ ]:
!cp best_model.hdf5 ./drive/My\ Drive/deepwriting/